In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists ECommerce_Lakehouse.gold_shipmode
(
    ShipMode_ID	Long	,
Ship_Mode	string	,
Created_TS	timestamp,
Modified_TS timestamp
)

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from ECommerce_Lakehouse.gold_shipmode").first()[0]

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 5, Finished, Available, Finished)

In [4]:
Max_Date

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 6, Finished, Available, Finished)

'1900-01-01'

In [5]:
df_bronze=spark.sql("select distinct Ship_Mode from ECommerce_Lakehouse.bronze_sales where Modified_TS>'{}'".format(Max_Date))

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 7, Finished, Available, Finished)

In [6]:
Max_ID=spark.sql("select coalesce(max(ShipMode_ID),0) from ECommerce_Lakehouse.gold_shipmode").first()[0]

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 8, Finished, Available, Finished)

In [7]:
df_final=df_bronze.withColumn("ShipMode_ID",monotonically_increasing_id()+Max_ID+1)
df_final.createOrReplaceTempView("ViewShipMode")

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 9, Finished, Available, Finished)

In [8]:
%%sql
merge into ECommerce_Lakehouse.gold_shipmode as gs 
using ViewShipMode as vs  
on gs.Ship_Mode=vs.Ship_Mode
when MATCHED then 
update set
Modified_TS= current_timestamp()

when not matched then
INSERT
(
gs.ShipMode_ID,
gs.Ship_Mode,
gs.Created_TS,
gs.Modified_TS
)
VALUES
(
vs.ShipMode_ID,
vs.Ship_Mode,
current_timestamp(),
current_timestamp()

)



StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [10]:
%%sql

SELECT * FROM gold_shipmode 

StatementMeta(, 0d3eb234-5bb6-45c9-a410-16f2335b4368, 12, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>